## Importing the libraries

## Device

PyTorch can use both CPU or GPU, depending on what is available:

## Dataset

The training dataset is composed of the inputs and required outputs of the neural network. In this case, the inputs are represented in a [tensor](https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html) of $4 \times 2$ dimension and the output (labels) in a $4 \times 1$ tensor.

Tensors are a specialized data structure that are very similar to arrays and matrices. In PyTorch, tensors are used to encode the inputs and outputs of a model, as well as the model’s parameters. We are also using a [`Dataset`](https://pytorch.org/docs/1.10.0/data.html#iterable-style-datasets) with the `DataLoader` to iterate the training set.

In [ ]:
from torch.utils.data import DataLoader

training_data = XorDataset()
train_dataloader = DataLoader(training_data, batch_size=1, shuffle=False)

## Neural Network

The neural network is defined by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

For the XOR network, we need two layers (refer to the slides).

The neural network is instanciated by creating the class and associating it to a processing device:

After the definition of the dataset and model architecture, we can train the network. We're using backpropagation, as depicted below:

We are using Min Square Error loss function and Stochastic Gradient Descent for training with 2000 epochs.

Let's plot the loss function:

We can now check the network parameters:

## Check the model

Let's check if the model behaves as expected: